In [4]:
# === Deel 2 pipeline runner (Jupyter-safe, één cel) ===
# Pas ALLEEN deze regel aan naar jouw CSV-bestand:
DATA_PATH = "college_statistics.csv"   # <--- verander hier zo nodig

# 1) (eenmalig) dependencies installeren – safe als het al bestaat
try:
    import pandas, numpy, scipy, statsmodels, sklearn, matplotlib, patsy  # noqa: F401
except Exception:
    %pip install pandas numpy scipy statsmodels scikit-learn matplotlib patsy

# 2) Lees het reeds aangemaakte script en vervang alleen de DATA_PATH-regel
from pathlib import Path
src_path = Path("/mnt/data/deel2_pipeline.py")
if not src_path.exists():
    # Schrijf een placeholder of jouw eigen pipeline code naar dit pad
    src_path.write_text(
        "# TODO: Voeg hier de code van deel2_pipeline.py toe\n"
        "def main():\n"
        "    print('Pipeline code ontbreekt. Voeg deze toe aan /mnt/data/deel2_pipeline.py')\n"
        "main()\n",
        encoding="utf-8"
    )

code = src_path.read_text(encoding="utf-8")
code = code.replace('DATA_PATH = "college_statistics.csv"', f'DATA_PATH = "{DATA_PATH}"')

# 3) Schrijf naar een tijdelijke runner en voer uit
runner_path = Path("/mnt/data/deel2_pipeline_run.py")
runner_path.write_text(code, encoding="utf-8")

# 4) Run (roept zelf main() aan)
%run -i /mnt/data/deel2_pipeline_run.py

print("\n✅ Klaar. Bekijk de map ./outputs (figures/, tables/, reports/summary.txt).")


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/deel2_pipeline.py'

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Deel 2 — Vragen 5a–5l: Generieke pipeline
Wijzig alleen DATA_PATH hieronder. De script detecteert automatisch kolommen,
bouwt modellen, voert diagnostiek uit, maakt grafieken en schrijft resultaten weg.

Vereiste packages:
  pip install pandas numpy scipy statsmodels scikit-learn matplotlib patsy

Uitvoer:
  ./outputs/
    - figures/*.png
    - tables/*.csv
    - reports/summary.txt
"""
import os
import math
import json
import warnings
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices, dmatrix
from scipy import stats
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.stats.diagnostic import het_breuschpagan, linear_reset
from statsmodels.stats.stattools import durbin_watson
from statsmodels.graphics.gofplots import qqplot
from statsmodels.graphics.regressionplots import plot_ccpr_grid

# -----------------------------
# === CONFIG ===
# -----------------------------
DATA_PATH = "college_statistics.csv"      # <<<<< Verander ALLEEN dit pad
TARGET = "Apps"                           # Doelvariabele
SEED = 123                                # Reproduceerbaarheid
N_TRAIN = 600                             # Grootte van estimation/train sample
N_FOLDS = 5                               # Voor cross-validation

# -----------------------------
# === OUTPUT MAP ===
# -----------------------------
OUT_DIR = "outputs"
FIG_DIR = os.path.join(OUT_DIR, "figures")
TAB_DIR = os.path.join(OUT_DIR, "tables")
REP_DIR = os.path.join(OUT_DIR, "reports")
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(TAB_DIR, exist_ok=True)
os.makedirs(REP_DIR, exist_ok=True)

warnings.filterwarnings("ignore", category=FutureWarning)
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 200)

# -----------------------------
# === HULPFUNCTIES ===
# -----------------------------
def safe_log(x: pd.Series, min_shift: float = 1e-6) -> pd.Series:
    """Log met kleine shift zodat 0/negatieven worden vermeden."""
    shift = max(min_shift, 1e-6)
    if (x <= 0).any():
        shift = max(shift, float(np.abs(x.min())) + 1e-6)
    return np.log(x + shift)

def duan_smearing_factor(residuals: np.ndarray) -> float:
    """Duan (1983) smearing factor voor terugtransformatie van log-model."""
    return float(np.mean(np.exp(residuals)))

def rmse(y_true, y_pred) -> float:
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def mae(y_true, y_pred) -> float:
    return float(mean_absolute_error(y_true, y_pred))

def numeric_and_categorical_cols(df: pd.DataFrame, target: str) -> Tuple[List[str], List[str]]:
    """Detecteer numerieke en categorische kolommen (exclusief target)."""
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if target in num_cols:
        num_cols.remove(target)
    cat_cols = df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
    return num_cols, cat_cols

def build_formula(target: str, num_cols: List[str], cat_cols: List[str]) -> str:
    """Bouw een patsy-formule met automatische categorische encoding."""
    rhs_terms = []
    rhs_terms += num_cols
    rhs_terms += [f"C({c})" for c in cat_cols]
    rhs = " + ".join(rhs_terms) if rhs_terms else "1"
    return f"{target} ~ {rhs}"

def backward_elimination(formula: str, df: pd.DataFrame, p_thresh: float = 0.05) -> sm.regression.linear_model.RegressionResultsWrapper:
    """Backward elimination op basis van p-waarden."""
    current_formula = formula
    while True:
        model = smf.ols(current_formula, data=df).fit()
        pvals = model.pvalues.drop("Intercept", errors="ignore")
        if pvals.empty:
            break
        worst = pvals.idxmax()
        worst_p = pvals.max()
        if worst_p <= p_thresh:
            break
        # variabele verwijderen uit formule
        terms = current_formula.split("~")[1].strip().split("+")
        terms = [t.strip() for t in terms if t.strip() != worst]
        new_rhs = " + ".join(terms) if terms else "1"
        current_formula = f"{TARGET} ~ {new_rhs}"
    return smf.ols(current_formula, data=df).fit()

def model_diagnostics(model, df: pd.DataFrame, target: str, prefix: str):
    """Voer diagnostiek uit (5e/5g) en schrijf plots + tabelletjes weg."""
    # Residuals en fitted
    resid = model.resid
    fitted = model.fittedvalues

    # 1) Normaliteit residuen (Shapiro) + QQ plot
    sh_p = stats.shapiro(resid).pvalue if len(resid) <= 5000 else np.nan
    fig = qqplot(resid, line="45")
    plt.title(f"QQ-plot residuen — {prefix}")
    plt.tight_layout()
    fig_path = os.path.join(FIG_DIR, f"{prefix}_qq_residuals.png")
    plt.savefig(fig_path, dpi=160)
    plt.close()

    # 2) Homoskedasticiteit (Breusch–Pagan)
    exog = model.model.exog
    bp_LM, bp_LMp, bp_F, bp_Fp = het_breuschpagan(resid, exog)

    # 3) RESET (lineaire specificatie)
    reset_p = linear_reset(model, power=2).pvalue

    # 4) Autocorrelatie (Durbin–Watson)
    dw = durbin_watson(resid)

    diag = {
        "shapiro_p": sh_p,
        "breusch_pagan_p": bp_LMp,
        "reset_p": reset_p,
        "durbin_watson": dw,
        "aic": model.aic,
        "bic": model.bic,
        "r2": model.rsquared,
        "adj_r2": model.rsquared_adj,
        "n_params": int(model.df_model) + 1
    }
    pd.DataFrame([diag]).to_csv(os.path.join(TAB_DIR, f"{prefix}_diagnostics.csv"), index=False)

    # 5) CCPR-grid
    try:
        fig = plot_ccpr_grid(model)
        plt.suptitle(f"CCPR-plots — {prefix}")
        plt.tight_layout()
        fig_path = os.path.join(FIG_DIR, f"{prefix}_ccpr_grid.png")
        plt.savefig(fig_path, dpi=160)
        plt.close()
    except Exception as e:
        with open(os.path.join(REP_DIR, f"{prefix}_ccpr_warning.txt"), "w") as f:
            f.write(str(e))

def cv_scores_log_model(df_train: pd.DataFrame, formula_log: str, target: str, n_folds: int = 5, random_state: int = 123) -> Dict[str, float]:
    """5-fold CV voor log-model — rapporteer RMSE/MAE op log- én Apps-schaal (met Duan)."""
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    rmse_log, mae_log, rmse_apps, mae_apps = [], [], [], []

    y_full = df_train[target].values
    for tr_idx, va_idx in kf.split(df_train):
        df_tr = df_train.iloc[tr_idx].copy()
        df_va = df_train.iloc[va_idx].copy()

        # Fit op log-schaal
        m = smf.ols(formula_log, data=df_tr).fit()
        # Duan smearing op train-residuen
        smear = duan_smearing_factor(m.resid.values)

        # Validatie-voorspellingen
        log_pred = m.predict(df_va)
        y_pred_apps = np.exp(log_pred) * smear
        y_true_apps = df_va[target].values

        rmse_log.append(rmse(np.log(y_true_apps + 1e-6), log_pred))
        mae_log.append(mae(np.log(y_true_apps + 1e-6), log_pred))
        rmse_apps.append(rmse(y_true_apps, y_pred_apps))
        mae_apps.append(mae(y_true_apps, y_pred_apps))

    return {
        "CV_RMSE_log_mean": float(np.mean(rmse_log)),
        "CV_RMSE_log_sd": float(np.std(rmse_log, ddof=1)),
        "CV_MAE_log_mean": float(np.mean(mae_log)),
        "CV_MAE_log_sd": float(np.std(mae_log, ddof=1)),
        "CV_RMSE_apps_mean": float(np.mean(rmse_apps)),
        "CV_RMSE_apps_sd": float(np.std(rmse_apps, ddof=1)),
        "CV_MAE_apps_mean": float(np.mean(mae_apps)),
        "CV_MAE_apps_sd": float(np.std(mae_apps, ddof=1)),
    }

def add_generic_transforms(df: pd.DataFrame) -> pd.DataFrame:
    """Voeg veilige, generieke transformaties/interacties toe als de kolommen bestaan (5i)."""
    out = df.copy()
    if "Top10perc" in out: out["Top10perc_sq"] = out["Top10perc"] ** 2
    if "Expend" in out: out["log_Expend"] = safe_log(out["Expend"])
    if {"PhD", "Outstate"}.issubset(out.columns):
        out["PhD_Outstate"] = out["PhD"] * out["Outstate"]
    return out

def print_and_write_summary(lines: List[str], path: str):
    txt = "\n".join(lines)
    print(txt)
    with open(path, "w", encoding="utf-8") as f:
        f.write(txt)

# -----------------------------
# === MAIN PIPELINE ===
# -----------------------------
def main():
    np.random.seed(SEED)

    # 5a — Data laden en normaliteit van TARGET beoordelen
    df = pd.read_csv(DATA_PATH)
    if TARGET not in df.columns:
        raise ValueError(f"TARGET '{TARGET}' niet gevonden in dataset. Kolommen: {list(df.columns)}")

    # Histogram en QQ-plot van TARGET
    plt.figure()
    plt.hist(df[TARGET].dropna(), bins=30)
    plt.title(f"Histogram — {TARGET}")
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, "5a_hist_target.png"), dpi=160)
    plt.close()

    fig = qqplot(df[TARGET].dropna(), line="45")
    plt.title(f"QQ-plot — {TARGET}")
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, "5a_qq_target.png"), dpi=160)
    plt.close()

    shapiro_p = stats.shapiro(df[TARGET].dropna()).pvalue if len(df) <= 5000 else np.nan
    pd.DataFrame([{"shapiro_p_target": shapiro_p}]).to_csv(os.path.join(TAB_DIR, "5a_shapiro_target.csv"), index=False)

    # 5b — Split estimation/test (reproduceerbaar)
    if len(df) < N_TRAIN + 1:
        raise ValueError(f"Te weinig observaties ({len(df)}). N_TRAIN={N_TRAIN} vereist.")
    df_train, df_test = train_test_split(df, train_size=N_TRAIN, random_state=SEED)
    pd.DataFrame([{"N_train": len(df_train), "N_test": len(df_test)}]).to_csv(os.path.join(TAB_DIR, "5b_split_sizes.csv"), index=False)

    # 5c — Basis lineair Apps-model (exclusief gevolgvariabelen Accept/Enroll indien aanwezig)
    forbid = {"Accept", "Enroll"}
    num_cols, cat_cols = numeric_and_categorical_cols(df_train, TARGET)
    num_cols = [c for c in num_cols if c not in forbid]
    cat_cols = [c for c in cat_cols if c not in forbid]
    formula_apps = build_formula(TARGET, num_cols, cat_cols)
    model_apps = smf.ols(formula_apps, data=df_train).fit()
    model_apps.save(os.path.join(REP_DIR, "5c_model_apps.pickle"))
    with open(os.path.join(REP_DIR, "5c_model_apps.txt"), "w", encoding="utf-8") as f:
        f.write(model_apps.summary().as_text())

    # 5d — Backward elimination op Apps-model
    model_apps_be = backward_elimination(formula_apps, df_train, p_thresh=0.05)
    with open(os.path.join(REP_DIR, "5d_model_apps_backward.txt"), "w", encoding="utf-8") as f:
        f.write(model_apps_be.summary().as_text())
    pd.DataFrame({"final_formula":[model_apps_be.model.formula]}).to_csv(os.path.join(TAB_DIR, "5d_final_formula_apps.csv"), index=False)

    # 5e — Aannamencheck (Apps BE-model)
    model_diagnostics(model_apps_be, df_train, TARGET, prefix="5e_apps_backward")

    # 5f — Log-transformatie TARGET
    df_train_log = df_train.copy()
    df_test_log = df_test.copy()
    df_train_log["log_" + TARGET] = safe_log(df_train[TARGET])
    df_test_log["log_" + TARGET]  = safe_log(df_test[TARGET])
    # Formule voor log(TARGET) met dezelfde predictors
    formula_log = model_apps_be.model.formula.replace(TARGET, "log_" + TARGET)
    model_log = smf.ols(formula_log, data=df_train_log).fit()
    with open(os.path.join(REP_DIR, "5f_model_log.txt"), "w", encoding="utf-8") as f:
        f.write(model_log.summary().as_text())

    # 5g — Aannamencheck (log-model)
    model_diagnostics(model_log, df_train_log, "log_" + TARGET, prefix="5g_log_model")

    # 5h — Cross-Validation (log-model)
    cv_res = cv_scores_log_model(df_train_log, formula_log, target=TARGET, n_folds=N_FOLDS, random_state=SEED)
    pd.DataFrame([cv_res]).to_csv(os.path.join(TAB_DIR, "5h_cv_log_model.csv"), index=False)

    # 5i — Transformaties & interacties en herfitten
    df_train_t = add_generic_transforms(df_train_log)
    df_test_t  = add_generic_transforms(df_test_log)

    # Bouw formule opnieuw inclusief eventuele nieuwe kolommen (alle numeriek + categorisch zoals eerder)
    num_cols_t, cat_cols_t = numeric_and_categorical_cols(df_train_t, TARGET)
    num_cols_t = [c for c in num_cols_t if c not in forbid]
    cat_cols_t = [c for c in cat_cols_t if c not in forbid]
    # Gebruik log-target
    formula_log_t = build_formula("log_" + TARGET, num_cols_t, cat_cols_t)
    model_log_t = smf.ols(formula_log_t, data=df_train_t).fit()
    with open(os.path.join(REP_DIR, "5i_model_log_transforms.txt"), "w", encoding="utf-8") as f:
        f.write(model_log_t.summary().as_text())

    # Vergelijk kernstatistieken
    comp = pd.DataFrame([
        {"Model":"Origineel log(Apps)", "Adj_R2": model_log.rsquared_adj, "AIC": model_log.aic, "BIC": model_log.bic},
        {"Model":"Met transformaties",  "Adj_R2": model_log_t.rsquared_adj, "AIC": model_log_t.aic, "BIC": model_log_t.bic},
    ])
    comp.to_csv(os.path.join(TAB_DIR, "5i_model_compare.csv"), index=False)

    # Testprestatie op Apps-schaal (met Duan smearing)
    smear = duan_smearing_factor(model_log.resid.values)
    smear_t = duan_smearing_factor(model_log_t.resid.values)

    log_pred_test = model_log.predict(df_test_t)
    log_pred_test_t = model_log_t.predict(df_test_t)

    pred_apps = np.exp(log_pred_test) * smear
    pred_apps_t = np.exp(log_pred_test_t) * smear_t
    rmse_test = rmse(df_test[TARGET].values, pred_apps)
    rmse_test_t = rmse(df_test[TARGET].values, pred_apps_t)

    pd.DataFrame([
        {"Model":"Origineel", "RMSE_test_apps": rmse_test},
        {"Model":"Verbeterd", "RMSE_test_apps": rmse_test_t},
    ]).to_csv(os.path.join(TAB_DIR, "5i_test_performance.csv"), index=False)

    # 5j — Belangrijkste predictoren (op basis van verbeterd log-model)
    coefs = model_log_t.params.sort_values(key=np.abs, ascending=False)
    top = coefs.head(12).rename("coef").to_frame()
    top.to_csv(os.path.join(TAB_DIR, "5j_top_coefficients.csv"))

    # 5k — Voorspellingen & intervallen (train + test), log- en Apps-schaal
    pred_train = model_log_t.get_prediction(df_train_t).summary_frame(alpha=0.05)  # mean_ci
    pred_test  = model_log_t.get_prediction(df_test_t).summary_frame(alpha=0.05)

    # Back-transform met smearing
    for frame, name in [(pred_train, "train"), (pred_test, "test")]:
        frame["pred_apps"] = np.exp(frame["mean"]) * smear_t
        frame["ci_lo_apps"] = np.exp(frame["mean_ci_lower"]) * smear_t
        frame["ci_hi_apps"] = np.exp(frame["mean_ci_upper"]) * smear_t
        frame.to_csv(os.path.join(TAB_DIR, f"5k_predictions_{name}.csv"), index=True)

    # Rapport: kernsamenvatting 5a–5l
    lines = []
    lines.append("=== Deel 2 — Samenvatting 5a–5l ===")
    lines.append(f"Dataset: {DATA_PATH}, N={len(df)}; Train={len(df_train)}; Test={len(df_test)}; Seed={SEED}")
    lines.append(f"5a Shapiro p (TARGET): {shapiro_p:.4g}")
    lines.append(f"5c Formula (Apps): {formula_apps}")
    lines.append(f"5d Final (Apps BE): {model_apps_be.model.formula}")
    lines.append(f"5f Log Formula: {formula_log}")
    lines.append(f"5i Log+Transforms Formula: {formula_log_t}")
    lines.append("---- Fit-statistieken ----")
    lines.append(f"Apps BE: R2={model_apps_be.rsquared:.3f}, AdjR2={model_apps_be.rsquared_adj:.3f}, AIC={model_apps_be.aic:.1f}, BIC={model_apps_be.bic:.1f}")
    lines.append(f"Log:     R2={model_log.rsquared:.3f},   AdjR2={model_log.rsquared_adj:.3f},   AIC={model_log.aic:.1f},   BIC={model_log.bic:.1f}")
    lines.append(f"Log+T:   R2={model_log_t.rsquared:.3f}, AdjR2={model_log_t.rsquared_adj:.3f}, AIC={model_log_t.aic:.1f}, BIC={model_log_t.bic:.1f}")
    lines.append("---- 5h Cross-Validation (log-model) ----")
    lines.append(json.dumps(cv_res, indent=2))
    lines.append("---- 5i Testprestatie (Apps-schaal, Duan) ----")
    lines.append(f"Origineel log RMSE_test={rmse_test:.2f}")
    lines.append(f"Verbeterd log RMSE_test={rmse_test_t:.2f}")
    print_and_write_summary(lines, os.path.join(REP_DIR, "summary.txt"))

    print("\nKlaar ✅  Bekijk de map ./outputs voor figuren, tabellen en rapporten.")

main()


SyntaxError: invalid syntax (<unknown>, line 1)